In [5]:
import numpy as np
import time

### In this exercise we expect you to demonstrate your ability to / knowledge of:
- optimization
- error debugging
- Performance improvement
- OOPS
- Git Actions

### Improve the efficiency of following code

In [6]:
starttime1=time.time()
total = 0
for i in np.arange(100000):
    total = i + total
endtime1=time.time()
f_time=endtime1-starttime1
print(total)
print(f'for loop method took {f_time}')

704982704
for loop method took 0.015994548797607422


C:\Users\N-Workstation\AppData\Local\Temp\ipykernel_449976\1055407611.py:4: RuntimeWarning: overflow encountered in long_scalars
  total = i + total


Optimized Code

In [7]:
#Np.sum can handle in optimized manner

starttime1=time.time()

total_v2 = 0
n = 100000
total_v2 = np.sum(np.arange(n))
endtime1=time.time()

f_time=endtime1-starttime1
print(f'for loop method took {f_time}')
print(total_v2)

for loop method took 0.0
704982704


### idenitify code issue

In [8]:
import pandas as pd
import io
import requests
url="https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [9]:
#whats woring with following code->goal is to create new column and set the value for 'Paraguay' to 10
c[c.Country=='Paraguay']['new_col']=10


C:\Users\N-Workstation\AppData\Local\Temp\ipykernel_449976\3670693239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[c.Country=='Paraguay']['new_col']=10


Solution Below

In [10]:
#Code handles above error i.e create col: new_col which has value: 10 for 'Country' column having value: Paraguay, rest of rows of new_col would have nulls
c.loc[c['Country'] == 'Paraguay', 'new_col'] = 10

In [11]:
c[c.Country=='Paraguay'].head()

,Country,Region,new_col
189,Paraguay,SOUTH AMERICA,10.0


In [12]:
c.head()

,Country,Region,new_col
0,Algeria,AFRICA,NaN
1,Angola,AFRICA,NaN
2,Benin,AFRICA,NaN
3,Botswana,AFRICA,NaN
4,Burkina,AFRICA,NaN


### Parllel processing

Normalize each row of 2d array (list) to vary between 0 and 1

make sure code can execute on multiple cpu's

input:[[2, 3, 4, 5], [6, 9, 10, 12], [11, 12, 13, 14], [21, 24, 25, 26]]

Output:[[0.0, 0.3333333333333333, 0.6666666666666666, 1.0], [0.0, 0.5, 0.6666666666666666, 1.0], [0.0, 0.3333333333333333, 0.6666666666666666, 1.0], [0.0, 0.6, 0.8, 1.0]]

Solution

In [13]:
import numpy as np

# Sample 2D array of lists
data = np.array([[2, 3, 4, 5], [6, 9, 10, 12], [11, 12, 13, 14], [21, 24, 25, 26]])

# Min-Max scaling to normalize the data between 0 and 1
min_vals = np.min(data, axis=1, keepdims=True)
max_vals = np.max(data, axis=1, keepdims=True)

normalized_data = (data - min_vals) / (max_vals - min_vals)

print(normalized_data.tolist())

[[0.0, 0.3333333333333333, 0.6666666666666666, 1.0], [0.0, 0.5, 0.6666666666666666, 1.0], [0.0, 0.3333333333333333, 0.6666666666666666, 1.0], [0.0, 0.6, 0.8, 1.0]]


With Multiple CPUs

In [20]:
data = [[2, 3, 4, 5], [6, 9, 10, 12], [11, 12, 13, 14], [21, 24, 25, 26]]

import numpy as np
import multiprocessing as mp

def normalize_row(row):
    min_val = min(row, axis=1, keepdims=True)
    max_val = max(row, axis=1, keepdims=True)
    return [(val - min_val) / (max_val - min_val) for val in row]

# Normalize each row of the 2D array between 0 and 1 using multiple CPUs
def normalize_2d_array(data):
    num_processes = mp.cpu_count()
    pool = mp.Pool(processes=num_processes)
    normalized_data = pool.map(normalize_row, data)
    pool.close()
    pool.join()
    return normalized_data

In [21]:
output_data = normalize_2d_array(data)

print(output_data)

### OOPS-github actions

Following is the table structure

CREATE TABLE author(
    A_ID int NOT NULL,
    Name varchar(100),
    PRIMARY KEY(A_ID )
)

CREATE TABLE books(
   B_ID int NOT NULL PRIMARY KEY,
   Name varchar(100),
   Price int NOT NULL,
   A_ID int FOREIGN KEY REFERENCES author(A_ID)
); 


- Program should read ddl statements and parse column names, data types and length of the columns and constraints

- Program should generate required number of rows for parent and child tables  (can be taken as a parameter)

- Generated data of both tables should follow the normalization rules and also foreign key constraints and data types

- write approriate test cases wtih python test framework(unit test or pytest)

- code should be committed to Git only when all the testcases are passd (use git actions) , this step acts as a ci/cd step.

- Code should be using oops concets and pep 8 standards with proper documentation

- result data should be saved in parquet format.

Solution below:

I have attached following three files along with this jupyter notebook as part of email, Please find below files with description for each of them.

1) Below Cells contains Class which takes care of Creating db, Creating table, Inserting data, fetching data using pandas, writing to Parquet format.
2) Class file for above point 1 logic. (has same code)
3) test_file_forclass.py - code for pytest test cases.
3) pythonapp.yml - yaml file for github actions to check if the code is good for commit. (tested on github)


In [14]:
import sqlite3
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

class SQLiteParquetManager:
    def __init__(self, db_file):
        self.db_file = db_file

    def create_table(self, table_name, columns):
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        # Create the table using the provided columns
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns)})"
        cursor.execute(create_table_query)

        conn.commit()
        conn.close()

    def insert_data(self, table_name, data):
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        # Insert data into the table
        insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * len(data))})"
        cursor.execute(insert_query, data)

        conn.commit()
        conn.close()

    def retrieve_data_save_parquet(self, table_name,output_file):
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        # Retrieve all data from the table
        cursor.execute(f"SELECT * FROM {table_name}")
        data = cursor.fetchall()

        
        
        # Assuming the data contains column names (replace with actual column names if needed)
        column_names = [description[0] for description in cursor.description]
        df = pd.DataFrame(data, columns=column_names)
        print(df.head(10))

        # Save the DataFrame to a Parquet file
        pq.write_table(pa.Table.from_pandas(df), output_file)
        if df.empty is False:
            print('data succesfully written to parquet format')
        conn.close()

In [15]:
db_file = "my_database.db"
manager = SQLiteParquetManager(db_file)

# Create author table
table_name = "author"
columns = ["A_ID INTEGER PRIMARY KEY","Name VARCHAR(100)"]
manager.create_table(table_name, columns)


authors_name =[(1, 'Jane Austen'), (2, 'J.K. Rowling'), (3, 'Leo Tolstoy'), (4, 'Mark Twain'), (5, 'Harper Lee'), (6, 'William Shakespeare'), (7, 'Agatha Christie'), (8, 'Stephen King'), (9, 'George Orwell'), (10, 'Charles Dickens')]

# write data to author table
for i in authors_name:
    manager.insert_data(table_name, i)

# Retrieve data and save as Parquet
output_parquet_file = table_name + ".parquet"
retrieved_data = manager.retrieve_data_save_parquet(table_name,output_parquet_file)


   A_ID                 Name
0     1          Jane Austen
1     2         J.K. Rowling
2     3          Leo Tolstoy
3     4           Mark Twain
4     5           Harper Lee
5     6  William Shakespeare
6     7      Agatha Christie
7     8         Stephen King
8     9        George Orwell
9    10      Charles Dickens
data succesfully written to parquet format


In [16]:
db_file = "my_database.db"
manager = SQLiteParquetManager(db_file)

# Create author table
table_name = "books"
columns = ["B_ID int NOT NULL PRIMARY KEY",
        "Name varchar(100)",
        "Price int NOT NULL",
        "A_ID int",
        "FOREIGN KEY (A_ID) REFERENCES author(A_ID)"]
manager.create_table(table_name, columns)


books_data = [
    (101, 'Book 1', 25, 1),
    (102, 'Book 2', 30, 2),
    (103, 'Book 3', 20, 3),
    (104, 'Book 4', 15, 1),
    (105, 'Book 5', 28, 4),
    (106, 'Book 6', 22, 5),
    (107, 'Book 7', 35, 2),
    (108, 'Book 8', 18, 6),
    (109, 'Book 9', 40, 1),
    (110, 'Book 10', 27, 7),
]

# write data to author table
for i in books_data:
    manager.insert_data(table_name, i)

# Retrieve data and save as Parquet
output_parquet_file = table_name + ".parquet"
retrieved_data = manager.retrieve_data_save_parquet(table_name,output_parquet_file)


   B_ID     Name  Price  A_ID
0   101   Book 1     25     1
1   102   Book 2     30     2
2   103   Book 3     20     3
3   104   Book 4     15     1
4   105   Book 5     28     4
5   106   Book 6     22     5
6   107   Book 7     35     2
7   108   Book 8     18     6
8   109   Book 9     40     1
9   110  Book 10     27     7
data succesfully written to parquet format
